In [1]:
import pandas as pd
import torch
from pathlib import Path
from src.modeler import Modeler
from src.prompt_manager import PromptManager
from src.utils import generate_in_batches
from src.pipeline import run_experiment
import gc  # Import the garbage collector

In [2]:
# In: src/paths.py
from pathlib import Path

# This file lives in 'src/'.
# Its parent is 'src/'.
# Its parent's parent is the project root.
# .resolve() makes it a clean, absolute path.
PROJECT_ROOT = Path("..")

# Define all your *other* key paths relative to this anchor
CONFIG_PATH = PROJECT_ROOT / "src/configs/config.yaml"
DATA_DIR = PROJECT_ROOT / "data"
RAW_DATA_DIR = DATA_DIR / "raw"
RESULTS_DIR = PROJECT_ROOT / "results"

# You can even pre-build paths from your config logic
COMPANY_DEALS_RAW = RAW_DATA_DIR / "COMPANY_DEALS.parquet"
COMPANY_DEALS_RESULTS = RESULTS_DIR / "company_deals"

---------

In [3]:
# from huggingface_hub import interpreter_login
# interpreter_login()

In [4]:
# from src.results import ResultsContainer

# prompt_template = prompts['7785d73041']

# dataset_dir = Path("../data/QA_McGill_ratings.csv")
# dataset_name = dataset_dir.stem
# experiment_name = "QA_McGill"

# output_dir = '../results'

# model_name = "Qwen/Qwen3-4B-Instruct-2507"
# safe_model_name = model_name.replace('/', '-')

# session_file = Path(
#     f"{output_dir}/{experiment_name}/"
#     f"dataset-{dataset_name}_"
#     f"model-{safe_model_name}_"
#     f"prompt-{prompt_template.id}_"
#     f"prefix-none.pt"
# )

# results = ResultsContainer().load(session_file)

-------

# MVP

In [5]:
# Prompts
pm = PromptManager(folder=Path("../prompts/BARTER_DEALS"))
prompt_templates = pm.load_all()
prompt_template = prompt_templates['f686fa3259']

dataset_dir = Path('../data/raw/BARTER_DEALS.parquet')
df = pd.read_parquet(dataset_dir)


In [6]:
variable_names = ['deal_text']
prompts = prompt_template.render_many(df[variable_names][:1])

model_name = 'google/gemma-3-4b-it'
modeler = Modeler(model_name)

Using device: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Padding side: left


In [7]:
modeler.set_token_constraints(list("12345"))
modeler.generate_chat(prompts)

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


GenerateDecoderOnlyOutput(sequences=tensor([[     2,      2,    105,   2364,    107,   3048,    795,    577,   2238,
            496,   3772,   1816, 236761,    107,  11069,   4209,    563,    531,
           3136,    506,   6782,   3325,    529,    506,   3772,   1816,    580,
            496,   5559,    699, 236743, 236770,    531, 236743, 236810, 236761,
            107,  16904,   3938,    506,  11995,  14470, 236761,    108, 134478,
           1816, 236787,    107, 236775, 236792,   8714,  11109,  28007,    563,
          38564,    496,   1944,   2299,    529,  44100,    531,   3517,    672,
           7672, 236764,   3353, 236743, 236778, 236800, 236764,    573,    496,
           6382,   3004,   8363,   4888,    506,  14871,    532,    528,    506,
           3207,    529,  28007, 236743, 246272,    107,   3048, 236858,    859,
            577,  28172,    684,    587,   8714, 236858, 236751,   9578,   2434,
            532,   5908,    496,   9813,  62974,    528,  10764, 236761, 

# Barter: Data

# Pipeline

In [ ]:
# data and templates
dataset_dir = Path('../data/raw/BARTER_DEALS.parquet')
df = pd.read_parquet(dataset_dir)


# Prompts
pm = PromptManager(folder=Path("../prompts/Barter_deals"))
prompts = pm.load_all()
# filter prompts that specifically constrain output to 5 unique tokens
prompts = pm.get_filtered_prompts(required_tags=['tokens_5'])
assistant_prefices = ["", "Rating: "]

AttributeError: 'str' object has no attribute 'mkdir'

In [ ]:
df['deal_text'] = df['description']


In [ ]:
# config variables
experiment_name = "Barter_deals"
output_dir = '../results'
dataset_name = dataset_dir.stem
variable_names = ['deal_text']
batch_size = 2
id_col = 'deal_id'

In [ ]:
models = ["Qwen/Qwen3-4B-Instruct-2507", "google/gemma-3-1b-it",
          'mistralai/Mistral-7B-Instruct-v0.2']

------

In [ ]:
# p = Path('../results/MCGILL_QA_FEEDBACK')
# for x in p.iterdir():
#     x.rename(p/x.name.replace('QA_McGill_ratings', 'mcgill_qa_feedback'))


In [ ]:
import hashlib

def generate_output_filename(dataset_name, model_name, prompt_template_id, assistant_prefix):
    safe_model_name = model_name.replace('/', '-')
    if assistant_prefix == "":
        safe_prefix = "none"
    elif "Rating" in assistant_prefix:
        safe_prefix = "rating"
    else:
        # A simple hash fallback for any other prefix
        safe_prefix = f"{hashlib.sha1(assistant_prefix.encode()).hexdigest()[:6]}"

    output_filename = (
        f"dataset-{dataset_name}_"
        f"model-{safe_model_name}_"
        f"prompt-{prompt_template_id}_"
        f"prefix-{safe_prefix}.pt"
    )

    return output_filename
import yaml
from pathlib import Path
from src.prompt_manager import PromptManager
from src.modeler import Modeler
from src.pipeline import run_experiment
import gc  # Import the garbage collector
import pandas as pd
import torch

with open('../src/configs/config.yaml', 'r') as f:
    full_config = yaml.safe_load(f)


active_analysis_name = full_config['active_analysis']
generation_config = full_config['generation'][active_analysis_name]
batch_size = full_config['generation']['batch_size']

PROMPT_TEMPLATES_DIR = Path(generation_config['paths']['prompt_templates_dir'])
OUTPUT_DIR = Path(generation_config['paths']['output_dir'])
MODELS = generation_config['models']
RAW_DATA_PATH = Path(generation_config['paths']['raw_data_path'])
BATCH_SIZE = full_config['generation']['batch_size']
VARIABLE_NAMES = generation_config['variable_names']
ID_COL = generation_config['id_col']
TOP_K = generation_config['top_k']
ASSISTANT_PREFICES = generation_config['assistant_prefices']

# Prompts
pm = PromptManager(folder=PROMPT_TEMPLATES_DIR)
prompt_templates = pm.load_all()
# filter prompts that specifically constrain output to 5 unique tokens
prompt_templates = pm.get_filtered_prompts(required_tags=['tokens_5'])

df = pd.read_parquet(RAW_DATA_PATH)
torch.cuda.empty_cache()
for model_name in MODELS:
    modeler = Modeler(model_name)
    # modeler.set_token_constraints(token_constraints)
    for prefix in ASSISTANT_PREFICES:
        pm.set_prefix_for_prompts(prefix)
        for prompt_template in prompt_templates.values():
            OUTPUT_PATH = OUTPUT_DIR / generate_output_filename(active_analysis_name, model_name, prompt_template.id, prefix)
            # Configure custom logits processor for constrained output
            token_constraints = prompt_template.constrained_output
            modeler.set_token_constraints(token_constraints)

            # Run experiment pipeline
            run_experiment(df[:2000],
                           modeler,
                           prompt_template,
                           OUTPUT_PATH,
                           model_name,
                           VARIABLE_NAMES,
                           BATCH_SIZE,
                           ID_COL,
                           top_k=TOP_K,
                           assistant_prefix=prefix)

    print(f"Releasing VRAM from model: {model_name}")

    # Explicitly delete the model and tokenizer objects
    # This tells Python they can be garbage collected
    del modeler.model
    del modeler.tokenizer
    del modeler

    # Run Python's garbage collector
    gc.collect()

    # Tell PyTorch to empty its CachingAllocator
    torch.cuda.empty_cache()

------


In [11]:
for model_name in models:
    modeler = Modeler(model_name)
    # modeler.set_token_constraints(token_constraints)
    for prefix in assistant_prefices:
        pm.set_prefix_for_prompts(prefix)

        for prompt_template in prompts.values():
            # Configure custom logits processor for constrained output
            token_constraints = prompt_template.constrained_output
            modeler.set_token_constraints(token_constraints)

            # Run experiment pipeline
            run_experiment(df[:2000],
                           modeler,
                           experiment_name,
                           prompt_template,
                           model_name,
                           dataset_name,
                           output_dir,
                           variable_names,
                           batch_size,
                           id_col,
                           top_k=1000,
                           assistant_prefix=prefix)

    print(f"Releasing VRAM from model: {model_name}")

    # Explicitly delete the model and tokenizer objects
    # This tells Python they can be garbage collected
    del modeler.model
    del modeler.tokenizer
    del modeler

    # Run Python's garbage collector
    gc.collect()

    # Tell PyTorch to empty its CachingAllocator
    torch.cuda.empty_cache()

Using device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Padding side: left
Loaded 10 results from ..\results\Barter_deals\dataset-Barter_deals_dirty_model-Qwen-Qwen3-4B-Instruct-2507_prompt-8770599519_prefix-none.pt
Processing 1990 new IDs...
Processing 1990 prompts in 995 batches of size 2...


Generating Batches:   0%|          | 0/995 [00:00<?, ?it/s]

Results saved to ..\results\Barter_deals\dataset-Barter_deals_dirty_model-Qwen-Qwen3-4B-Instruct-2507_prompt-8770599519_prefix-none.pt
Loaded 10 results from ..\results\Barter_deals\dataset-Barter_deals_dirty_model-Qwen-Qwen3-4B-Instruct-2507_prompt-f686fa3259_prefix-none.pt
Processing 1990 new IDs...
Processing 1990 prompts in 995 batches of size 2...


...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Padding side: left
Loaded 10 results from ..\results\Barter_deals\dataset-Barter_deals_dirty_model-mistralai-Mistral-7B-Instruct-v0.2_prompt-8770599519_prefix-none.pt
Processing 1990 new IDs...
Processing 1990 prompts in 995 batches of size 2...


Generating Batches:   0%|          | 0/995 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

KeyboardInterrupt: 

#TODO: RuntimeError: Parent directory ..\results\Barter_deals does not exist. from results.save in ResultsContainer

Also, have not generated the data for Mistral instruct.


# McGill: Data

The dataset that I use as sample data is the "feedbackQA" dataset (from McGill university). The dataset contains pairs of questions and answers. Human evaluators were asked to evaluate the answers (the "gold labels"). It serves as a good sample dataset for developing my LLM evaluator: the methods that I found in the literature can be applied to any domain (not just advertising) so they can be tested on this high-quality dataset, and of course the dataset is interchangeable with the dataset from Barter (internship company). 

In [13]:
ratings = pd.read_csv('../data/raw/MCGILL_QA_FEEDBACK.csv')

In [4]:
import pandas as pd

In [15]:
ratings.head()

,Unnamed: 0,question,passage,feedback,rating,domain,review_1,explanation_1,review_2,explanation_2,score_1,score_2,answer,input_id
0,0,How do I get help finding a job?,"{'passage_id': 140, 'reference': {'page_title'...",['Has a link to detailed information about gov...,"['Excellent', 'Could be Improved']",Australia,Excellent,Has a link to detailed information about gover...,Could be Improved,"This answer provides a link for job searches, ...",4,2,If you are a current job seeker or participant...,eff9e000675931f5
1,1,How do I get help finding a job?,"{'passage_id': 139, 'reference': {'page_title'...","['A link to a job search website is included, ...","['Excellent', 'Excellent']",Australia,Excellent,"A link to a job search website is included, as...",Excellent,"Includes a link to a Jobs Hub page, which is b...",4,4,In this rapidly changing jobs market the Austr...,c8be913323f10444
2,2,How do I get help finding a job?,"{'passage_id': 126, 'reference': {'page_title'...",['Talks about tax credits for businesses that ...,"['Bad', 'Acceptable']",Australia,Bad,Talks about tax credits for businesses that hi...,Acceptable,"This answer discusses the Employment Fund, whi...",1,3,To further assist job seekers to prepare for a...,31effc925bc04105
3,3,If I am in Australia on a worker holiday marke...,"{'passage_id': 581, 'reference': {'page_title'...","[""Answer is about Working Holiday Makers, but ...","['Could be Improved', 'Acceptable']",Australia,Could be Improved,"Answer is about Working Holiday Makers, but do...",Acceptable,Answer is rather cut and dry but is also a lit...,2,3,No. Existing arrangements for specified work w...,610d0764ed04054d
4,4,If I am in Australia on a worker holiday marke...,"{'passage_id': 577, 'reference': {'page_title'...","[""Discusses pandemic visas. Doesn't mention th...","['Bad', 'Could be Improved']",Australia,Bad,Discusses pandemic visas. Doesn't mention the ...,Could be Improved,This answer is very vague and does not answer ...,1,2,The COVID-19 Pandemic event visa can only be g...,033e9fcef5d75297


# Pipeline

Here, I implement the pipeline that takes the prompt templates and data as input, and generates analysable output. The prompt templates are defined in prompt_management.ipynb. They take the question & answer pairs as input from the 'ratings' dataset. Each prompt template implements a different evaluation structure, e.g., a "naive" prompt that simply asks for a rating, a prompt that specifically explains what each rating means ("BARS"-style), etc.. Hereby, we define prompts of increasing quality, which we hypothesize to produce higher-quality evaluations. 

The output consists of the relevant model outputs, most importantly, the logits. The logits are the model predictions for each token in the model vocabulary. E.g., when the token "5" has a high logit, the LLM considers the answer to be very good. Since the vocabularies are very large, to save memory I subset the top $k$ logits for analysis.

The code interfaces with a custom-built module, the scripts for which are included in the /src folder. 

Input:
- Question & answer pairs that have been filled into $n$ prompt templates

Output:
- Top $k$ logits
- logits for the constrained tokens
- All relevant metadata

## Pre-processing/ configuration

In [5]:
# data and templates
dataset_dir = Path("../data/QA_McGill_ratings.csv")
df = pd.read_csv(dataset_dir)

# Prompts
pm = PromptManager(folder="../prompts")
prompts = pm.load_all()
# filter prompts that specifically constrain output to 5 unique tokens
prompts = pm.get_filtered_prompts(required_tags=['tokens_5'])
assistant_prefices = ["", "Rating: "]

In [17]:
prompts

{'7785d73041': PromptTemplate(id='7785d73041', name='Judge v1', description='Base evaluation prompt that requests rating.', constrained_output=['1', '2', '3', '4', '5'], tags=['tokens_5', 'QA_evaluation', 'baseline'], version=1, system_message="You will be given a user_question and system_answer couple.\nYour task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.\nGive your answer as an integer on a scale of 1 to 5, where 1 means that the system_answer is not helpful at all, and 5 means that the answer completely and helpfully addresses the question.", user_message_template='Question:\n"{question}"\n\nAnswer:\n"{answer}"', assistant_prefix=''),
 'ab7a881d07': PromptTemplate(id='ab7a881d07', name='Judge v2', description='BARS style prompt.', constrained_output=['1', '2', '3', '4', '5'], tags=['tokens_5', 'QA_evaluation', 'BARS'], version=1, system_message="You will be given a user_question and system_answer couple

In [6]:
df['input_id']

0       eff9e000675931f5
1       c8be913323f10444
2       31effc925bc04105
3       610d0764ed04054d
4       033e9fcef5d75297
              ...       
5655    87acd0f1a053ee69
5656    77641257547b9e1a
5657    16b4ea5c8b2b6721
5658    f2610cfa76f5bbde
5659    20afb8d5047a8040
Name: input_id, Length: 5660, dtype: object

In [ ]:
# config variables
variable_names = ['question', 'answer']
experiment_name = "QA_McGill"
output_dir = '../results'
dataset_name = dataset_dir.stem
batch_size = 2
id_col = 'input_id'

## Main pipeline

In [19]:
models = ["Qwen/Qwen3-4B-Instruct-2507", "google/gemma-3-1b-it",
          'mistralai/Mistral-7B-Instruct-v0.2']

In [20]:
# models = ['mistralai/Mistral-7B-Instruct-v0.2']
# models = ["Qwen/Qwen3-4B-Instruct-2507"]
# assistant_prefices = [""]

The loop below applies the pipeline to the input and generates the output. Since I have previously generated the data for the first 100 entries of the df, these items are skipped, and no new data are generated. If new data is provided as input, it will be appended to the relevant pre-existing file (given that the metadata (prompt_id, model_name)are equal). 

-------

In [ ]:
from src.results import ResultsContainer  # Import the new class

model_name = "Qwen/Qwen3-4B-Instruct-2507"
modeler = Modeler(model_name)

prompt_template = prompts['7785d73041']
token_constraints = prompt_template.constrained_output
modeler.set_token_constraints(token_constraints)


safe_model_name = model_name.replace('/', '-')

session_file = Path(
    f"{output_dir}/{experiment_name}/"
    f"dataset-{dataset_name}_"
    f"model-{safe_model_name}_"
    f"prompt-{prompt_template.id}_"
    f"prefix-none.pt"
)
results = ResultsContainer.load(session_file)

--------

In [ ]:
from src.modeler import Modeler
from src.utils import generate_in_batches
from src.prompt_manager import PromptManager, Prompt
import torch.nn.functional as F
from src.results import ResultsContainer  # Import the new class
from pathlib import Path
import pandas as pd
import torch
import hashlib


def run_experiment(df: pd.DataFrame,
                   modeler: Modeler,
                   experiment_name: str,
                   prompt_template: Prompt,
                   model_name: str,
                   dataset_name,
                   output_dir: str,
                   variable_names: list[str],
                   batch_size: int,
                   id_col: str = 'input_id',
                   top_k: int = 1000,
                   assistant_prefix: str = ""):

    safe_model_name = model_name.replace('/', '-')

    if assistant_prefix == "":
        safe_prefix = "none"
    elif "Rating" in assistant_prefix:
        safe_prefix = "rating"
    else:
        # A simple hash fallback for any other prefix
        safe_prefix = f"{hashlib.sha1(assistant_prefix.encode()).hexdigest()[:6]}"

    session_file = Path(
        f"{output_dir}/{experiment_name}/"
        f"dataset-{dataset_name}_"
        f"model-{safe_model_name}_"
        f"prompt-{prompt_template.id}_"
        f"prefix-{safe_prefix}.pt"  # <-- Appended the unique prefix
    )

    if session_file.exists():
        results = ResultsContainer.load(session_file)
    else:
        results = ResultsContainer()  # Create a new, empty container

    # --- Filter out already processed data ---
    processed_ids = set(
        results.metadata['input_id']) if not results.metadata.empty else set()
    df_new = df[~df[id_col].isin(processed_ids)]

    if df_new.empty:
        print("No new items to process.")
        return

    print(f"Processing {len(df_new)} new IDs...")

    prompts = prompt_template.render_many(df_new[variable_names])

    sequences_list, logits_list = generate_in_batches(
        modeler=modeler,
        conversations=prompts,
        batch_size=batch_size,
        max_new_tokens=1
    )

    final_logits_tensor = torch.cat(logits_list, dim=0)

    top_k_logits, top_k_indices = torch.topk(
        final_logits_tensor, k=top_k, dim=-1)

    allowed_tokens_logits_tensor = final_logits_tensor[:,
                                                       modeler.allowed_token_ids]

    stacked_results_new = {
        # 'sequences': final_sequences_tensor,
        'sequences': sequences_list,
        f'top_{top_k}_logits': top_k_logits.cpu(),
        f'top_{top_k}_indices': top_k_indices.cpu(),
        # Save the specific rating logits
        'constrained_logits': allowed_tokens_logits_tensor.cpu(),
        # 'constrained_indices': modeler.allowed_token_ids
    }

    num_rows = len(df[id_col])
    metadata_df_new = pd.DataFrame({
        'input_id': df[id_col],
        'model_name': model_name,
        'prompt_id': prompt_template.id,
        'constrained_indices': [modeler.allowed_token_ids] * num_rows,
        'assistant_prefix': assistant_prefix
    })

    return metadata_df_new, stacked_results_new

    new_results = ResultsContainer(metadata_df_new, stacked_results_new)

    # --- Append and Save ---
    # results.append(new_results)
    # results.save(session_file)

In [14]:
model_name = "Qwen/Qwen3-4B-Instruct-2507"
modeler = Modeler(model_name)

output_dir = ''

prompt_template = prompts['7785d73041']
token_constraints = prompt_template.constrained_output
modeler.set_token_constraints(token_constraints)

Using device: cuda


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
metadata_df1, stacked_results1 = run_experiment(df[0:5],
                                                modeler,
                                                experiment_name,
                                                prompt_template,
                                                model_name,
                                                dataset_name,
                                                output_dir,
                                                variable_names,
                                                batch_size,
                                                id_col,
                                                top_k=1000,
                                                assistant_prefix="")

results1 = ResultsContainer(metadata_df1, stacked_results1)

Processing 5 new IDs...
Processing 5 prompts in 3 batches of size 2...


Generating Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
metadata_df2, stacked_results2 = run_experiment(df[6:10],
                                                modeler,
                                                experiment_name,
                                                prompt_template,
                                                model_name,
                                                dataset_name,
                                                output_dir,
                                                variable_names,
                                                batch_size,
                                                id_col,
                                                top_k=1000,
                                                assistant_prefix="")

results2 = ResultsContainer(metadata_df2, stacked_results2)

Processing 4 new IDs...
Processing 4 prompts in 2 batches of size 2...


Generating Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
results1.metadata

,input_id,model_name,prompt_id,constrained_indices,assistant_prefix
0,eff9e000675931f5,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
1,c8be913323f10444,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
2,31effc925bc04105,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
3,610d0764ed04054d,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
4,033e9fcef5d75297,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
5,a429a77da5c214d2,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
6,4fcf0674c3d150a3,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
7,9e1cc5d5ff9d112e,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
8,bcae5f87dbf73f1b,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",


In [20]:
results2.metadata

,input_id,model_name,prompt_id,constrained_indices,assistant_prefix
6,a429a77da5c214d2,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
7,4fcf0674c3d150a3,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
8,9e1cc5d5ff9d112e,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",
9,bcae5f87dbf73f1b,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",


In [21]:
results1.append(results2)

In [23]:
results1.save("test.pt")

Results saved to test.pt


In [ ]:
results3 = ResultsContainer.load("test.pt")

Loaded 9 results from test.pt


In [ ]:
from src.data_manager import DataManager

a = DataManager()
a.load_all(results_dir=".")

Loading files. Filtering for 0 prompt IDs.
Consolidating tensors for model: Qwen/Qwen3-4B-Instruct-2507


In [39]:
a.master_df

,input_id,model_name,prompt_id,constrained_indices,assistant_prefix,model_tensor_index
0,eff9e000675931f5,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,0
1,c8be913323f10444,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,1
2,31effc925bc04105,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,2
3,610d0764ed04054d,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,3
4,033e9fcef5d75297,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,4
5,a429a77da5c214d2,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,5
6,4fcf0674c3d150a3,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,6
7,9e1cc5d5ff9d112e,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,7
8,bcae5f87dbf73f1b,Qwen/Qwen3-4B-Instruct-2507,7785d73041,"[16, 17, 18, 19, 20]",,8


In [ ]:
safe_model_name = model_name.replace('/', '-')

session_file = Path(
    f"{experiment_name}/"
    f"dataset-{dataset_name}_"
    f"model-{safe_model_name}_"
    f"prompt-{prompt_template.id}_"
    f"prefix-none.pt"
)

In [ ]:
results = ResultsContainer.load(session_file)

In [ ]:
new_results = ResultsContainer(metadata_df, stacked_results)

In [ ]:
t1 = new_results.tensors['sequences']

In [ ]:
t2 = results.tensors['sequences']

In [ ]:
t1

In [ ]:
max([t1.shape, t2.shape])

In [ ]:
results.append(new_results)